# Quickstart tutorial de langchain
https://www.youtube.com/watch?v=LBNpyjcbv0o

In [6]:
# ! pip install langchain
# ! pip install langchain-openai
# ! pip install beautifulsoup4 ## Web scraping library
# ! pip install faiss-cpu ## Vector store and search library

In [2]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = ""
## Import llm 
llm = ChatOpenAI()

### Vamos a crear un chat capaz de extraer informacion de una web y procesarla
Para ello vamos a usar LangChain 0.1 y a encadenar varios eslabones (chain). 
Used Chains:
- document_chain
- retrieval_chain

#### Creacion de un retriever
Un retriever es una interfaz que devuelve documentos dada una query no estructurada.
Basicamente, estructura en documentos la informacion no estructurada (docs, webs, pdf..)

En este caso, se va a crear un retriever de una web de origen

In [12]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.vectorstores.faiss import VectorStore
from langchain.text_splitter import RecursiveCharacterTextSplitter

def getVectorStoreFromUrl(url) -> VectorStore:
  ## Extract data from a new source
  loader = WebBaseLoader(url)
  webInfo = loader.load()
  
  ## Transform docs to an embedding.
  ## OpenAI uses his own embeddings
  openAiEmbeddings = OpenAIEmbeddings()

  text_splitter = RecursiveCharacterTextSplitter()
  splittedWebInfo = text_splitter.split_documents(webInfo)
  # splittedDocs
  vectorStore = FAISS.from_documents(splittedWebInfo, openAiEmbeddings)
  return vectorStore

In [80]:
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever

def createRetrieverChainFromUrl(llm, prompt, url):
  document_chain = create_stuff_documents_chain(llm, prompt)

  vectorStore = getVectorStoreFromUrl(url)
  # retriever_chain = create_retrieval_chain(vectorStore.as_retriever(), document_chain)

  retriever_chat_chain = create_history_aware_retriever(llm, vectorStore.as_retriever(), prompt)
  
  return create_retrieval_chain(retriever_chat_chain, document_chain)

In [54]:
from langchain_core.messages import HumanMessage, AIMessage

def getInitialChatHistory():
  return [
    AIMessage(content="Hola! Soy el señor Meesecks!, la vida es dolor!, ¿En qué puedo ayudarte?")
    AIMessage(content="Hola! Soy el señor Meesecks!, la vida es dolor!, ¿En qué puedo ayudarte?")
  ]

In [81]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

## Creamos el prompt inicial.
prompt: ChatPromptTemplate = ChatPromptTemplate.from_messages([
  ("system", "Responde a la pregunta solo con el contexto proporcionado. Contexto:\n\n{context}"),
  MessagesPlaceholder(variable_name="chat_history"),
  ("user", "{input}")
])

## Create retrieval chain
## Convertimos el vector store a un retriever
url = "https://kokedamasconamor.com/como-hacer-tu-kokedama-paso-a-paso/"
retriever_chain = createRetrieverChainFromUrl(llm, prompt, url)

In [85]:
response = retriever_chain.invoke({
  "context": "{context}",
  "chat_history": getInitialChatHistory(),
  "input": "Dime paso a paso como hacer un kokedama"
})

In [67]:
response['answer']

'¡Claro! Aquí te explico paso a paso cómo hacer tu propio kokedama:\n\n1. Elige la planta: selecciona una planta de tamaño pequeño con maceta de unos 9 cm.\n2. Reúne los materiales: sustrato universal o turba, tierra de akadama, humus de lombriz, musgo, pulverizador, agua, tijeras, hilo y un palillo de bambú u objeto punzante.\n3. Limpia el musgo: humedece y limpia el musgo para que quede flexible.\n4. Mezcla los sustratos: combina los sustratos en un recipiente hasta formar una bola.\n5. Prepara la planta: saca la planta de la maceta, quita parte de la tierra y deja algunas raíces expuestas.\n6. Forma la bola: cubre las raíces con sustrato formando una bola compacta.\n7. Cubre con musgo: envuelve la bola con el musgo y sujétalo con hilo.\n8. Sujeta con hilo: envuelve el musgo con hilo alrededor de la bola.\n9. Hacer los ajustes finales: corta el hilo y ajusta los extremos para un acabado perfecto.\n\n¡Y listo! Ya tendrás tu propio kokedama listo para decorar tu hogar. ¡Espero que te s

'¡Claro! Aquí te explico paso a paso cómo hacer tu propio kokedama:\n\n1. Elige la planta: selecciona una planta de tamaño pequeño con maceta de unos 9 cm.\n2. Reúne los materiales: sustrato universal o turba, tierra de akadama, humus de lombriz, musgo, pulverizador, agua, tijeras, hilo y un palillo de bambú u objeto punzante.\n3. Limpia el musgo: humedece y limpia el musgo para que quede flexible.\n4. Mezcla los sustratos: combina los sustratos en un recipiente hasta formar una bola.\n5. Prepara la planta: saca la planta de la maceta, quita parte de la tierra y deja algunas raíces expuestas.\n6. Forma la bola: cubre las raíces con sustrato formando una bola compacta.\n7. Cubre con musgo: envuelve la bola con el musgo y sujétalo con hilo.\n8. Sujeta con hilo: envuelve el musgo con hilo alrededor de la bola.\n9. Hacer los ajustes finales: corta el hilo y ajusta los extremos para un acabado perfecto.\n\n¡Y listo! Ya tendrás tu propio kokedama listo para decorar tu hogar. ¡Espero que te sea de ayuda!'